#Case Study 1 : Collecting Data from Twitter

** Due Date: Sep. 23, before the class**

*------------

**TEAM Members:** Please EDIT this cell and add the names of all the team members in your team

    member 1
    
    member 2
    
    ...

**Required Readings:** 
* Chapter 1 and Chapter 9 of the book [Mining the Social Web](http://www.learndatasci.com/wp-content/uploads/2015/08/Mining-the-Social-Web-2nd-Edition.pdf) 
* The codes for [Chapter 1](http://bit.ly/1qCtMrr) and [Chapter 9](http://bit.ly/1u7eP33)


** NOTE **
* Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost.

*----------------------

In [16]:
import io
import sys
import time
import json
import twitter
#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Go to http://twitter.com/apps/new to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://dev.twitter.com/docs/auth/oauth for more information
    # on Twitter's OAuth implementation.

    CONSUMER_KEY = '7vTffANLujbBuC6Svf8WPHe1A'
    CONSUMER_SECRET ='2PKGjWO8yUHeMFjelqYiMN78wo5dNfFMKOHRjA0jkvjOqC9UdY'
    OAUTH_TOKEN = '2159266260-B9Oo1OHoluExvrirDOclBjNbhRMDM1Yga9FGzVg'
    OAUTH_TOKEN_SECRET = 'qQA7dLwuGat4G18ccrf5l7TuvEkKfWJnETxwLqLOgyzmZ'

    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)

    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary


def Get_Stream(topic_para, filename):
    count = 0
    print >> sys.stderr, 'Filtering the public timeline for track="%s"' % (topic_para,)
    # Returns an instance of twitter.Twitter
    twitter_api = oauth_login()

    # Reference the self.auth parameter
    twitter_stream = twitter.TwitterStream(auth=twitter_api.auth)

    # See https://dev.twitter.com/docs/streaming-apis
    stream = twitter_stream.statuses.filter(track=topic_para)
    # For illustrative purposes, when all else fails, search for Justin Bieber
    # and something is sure to turn up (at least, on Twitter)
    # for tweet in stream:
    #     print tweet['text']
    # Save to a database in a particular collection
    with io.open('/Users/Chan/Desktop/{0}.json'.format(filename),
                 'w', encoding='utf-8') as f:
        for tweet in stream:
            f.write(unicode(json.dumps(tweet, ensure_ascii=False))+'\n')
            count += 1
            if count > 10000:
                print 'The End'
                return

Get_Stream('iphone', 'Test')




length 85


#Problem 1: Sampling Twitter Data with Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

###Report some statistics about the tweets you collected 

*The topic of interest: < INSERT YOUR TOPIC HERE>


*The total number of tweets collected:  < INSERT THE NUMBER HERE>

*-----------------------

#Problem 2: Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [3]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
import io
import re
import twitter
import json
from collections import Counter
from prettytable import PrettyTable


def load_json(filename):
    with io.open('/Users/Chan/Desktop/{0}.json'.format(filename),
                 encoding='utf-8') as f:
        return f.readlines()


def tweetsFreq(statuses):
    assert len(statuses) != 0, "必须是有效的tweets!"
    # Get texts and hashtags collection.
    status_text = [status.get('text') for status in statuses if status.get('text') is not None]
    status_hashtag = []
    for status in statuses:
        if status.get('entities') is not None:
            if status.get('entities').get('hashtags') is not None:
                for hashtag in status.get('entities').get('hashtags'):
                    if hashtag.get('text') is not None:
                        status_hashtag.append(hashtag.get('text'))

    words = [word for text in status_text for word in text.split()]
    if len(words) == 0 or len(status_hashtag) == 0:
        print "Cannot extract text or hashtag from tweets!"
        raise Exception

    # Advanced cleaning
    # Clean hashtag and word with digit.
    pattern_raw = re.compile(r"\w+|\\(\w+|\d+)", re.S)
    pattern_unicode = re.compile(r"\\")

    lala = []
    for word in words:
        temp_word = re.match(pattern_raw, word)
        if temp_word is not None:
            if temp_word.group() != 'http':
                lala.append(temp_word.group().lower())
    #lala = [re.match(pattern_raw, word).group().lower()
    #       for word in words]
    temp_list = []
    temp_word = []
    for word in lala:
        temp_word = re.split(pattern_unicode, word)
        for item in temp_word:
            temp_list.append(item)
    words = temp_list

    # Create a pretty table for each item.
    word_freq_list = Counter(words)
    hashtag_freq_list = Counter(status_hashtag)
    for label, data in (('Words', words), ('Hashtags', status_hashtag)):
        pt = PrettyTable(field_names=[label, 'Count'])
        c = Counter(data)
        [pt.add_row(kv) for kv in c.most_common()[:30]]
        pt.align[label], pt.align['Count'] = 'l', 'r'

        print pt

tweets = load_json('Test')
statuses = []
count = 0
for tweet in tweets:
    count += 1
    statuses.append(json.loads(tweet))
tweetsFreq(statuses)


+---------+-------+
| Words   | Count |
+---------+-------+
| iphone  |   164 |
| rt      |    79 |
| a       |    30 |
| the     |    28 |
| que     |    26 |
| un      |    26 |
| de      |    23 |
| i       |    20 |
| 6s      |    20 |
| 6       |    19 |
| la      |    19 |
| for     |    19 |
| apple   |    16 |
| tu      |    14 |
| 9       |    14 |
| an      |    14 |
| new     |    13 |
| l       |    13 |
| and     |    13 |
| ios     |    13 |
| e       |    13 |
| 7       |    12 |
| android |    12 |
| to      |    11 |
| le      |    11 |
| ipad    |    11 |
| have    |    11 |
| o       |    11 |
| https   |    10 |
| my      |    10 |
+---------+-------+
+----------------+-------+
| Hashtags       | Count |
+----------------+-------+
| iphone         |     7 |
| iOS9           |     6 |
| ios9           |     4 |
| iPhone         |     3 |
| f1             |     2 |
| apple          |     2 |
| deal           |     2 |
| SelfieStick    |     2 |
| GoPro          |     

**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [4]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
def popTweets(statuses):
    retweets = [(status['retweet_count'],
                status['retweeted_status']['user']['screen_name'],
                status['text'])
                # For each status
                for status in statuses
                if status.has_key('retweeted_status') 
               ]
    # Create a pretty table.
    pt = PrettyTable(field_names=['Count', 'Screen Name', 'Text'])
    [pt.add_row(kv) for kv in sorted(retweets, reverse=True)[0:10]]
    pt.max_width['Text'] = 50
    pt.align= 'l'
    print pt
    
popTweets(statuses)


+-------+---------------+---------------------------------+
| Count | Screen Name   | Text                            |
+-------+---------------+---------------------------------+
| 15622 | BoyQuotations | RT @BoyQuotations: iPhone 7 😱  |
|       |               |                                 |
|       |               | http://t.co/LMB7UrhLHx          |
| 15622 | BoyQuotations | RT @BoyQuotations: iPhone 7 😱  |
|       |               |                                 |
|       |               | http://t.co/LMB7UrhLHx          |
| 15622 | BoyQuotations | RT @BoyQuotations: iPhone 7 😱  |
|       |               |                                 |
|       |               | http://t.co/LMB7UrhLHx          |
| 15622 | BoyQuotations | RT @BoyQuotations: iPhone 7 😱  |
|       |               |                                 |
|       |               | http://t.co/LMB7UrhLHx          |
| 15622 | BoyQuotations | RT @BoyQuotations: iPhone 7 😱  |
|       |               |                    

**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [5]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
def popHU(statuses):
    hashtags = [hashtag['text'] for status in statuses for hashtag in status['entities']['hashtags']]
    users = [user_mention['screen_name'] 
             for status in statuses 
             for user_mention in status['entities']['user_mentions']]
    
    for label, data in [('Hashtag', hashtags), ('Users Mentioned', users)]:
        pt = PrettyTable(field_names=[label, 'Count'])
        c = Counter(data)
        [pt.add_row(kv) for kv in c.most_common()[:10]]
        print pt
        
popHU(statuses)

+----------------+-------+
|    Hashtag     | Count |
+----------------+-------+
|     iphone     |   7   |
|      iOS9      |   6   |
|      ios9      |   4   |
|     iPhone     |   3   |
|       f1       |   2   |
|     apple      |   2   |
|      deal      |   2   |
|  SelfieStick   |   2   |
|     GoPro      |   2   |
| 俺のストーカー |   2   |
+----------------+-------+
+-----------------+-------+
| Users Mentioned | Count |
+-----------------+-------+
|  BoyQuotations  |   12  |
|    mardidupin   |   5   |
|    T0MLINSV0N   |   3   |
|    laularubia   |   3   |
|     screamau    |   2   |
|    cagritaner   |   2   |
|      ZDNet      |   2   |
|     SdillotP    |   2   |
|   gogirlenergy  |   2   |
|     Kehysi_A    |   2   |
+-----------------+-------+


*------------------------

#Problem 3: Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

Step 1, using the code from the book to fetch all follower IDs and friend IDs of "Fitness Motivation". Its screen name is "sb14body". It has 41 friends and 48.7 thousands followers. This section is to save 20 of friend IDs and 20 of follower IDs.

In [7]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
from functools import partial
from sys import maxint
import sys
import time
from urllib2 import URLError
from httplib import BadStatusLine


#Make twitter request
def make_twitter_request(twitter_api_func, max_errors=10, *args, **kw): 
    
    # A nested helper function that handles common HTTPErrors. Return an updated
    # value for wait_period if the problem is a 500 level error. Block until the
    # rate limit is reset if it's a rate limiting issue (429 error). Returns None
    # for 401 and 404 errors, which requires special handling by the caller.
    def handle_twitter_http_error(e, wait_period=2, sleep_when_rate_limited=True):
    
        if wait_period > 3600: # Seconds
            print >> sys.stderr, 'Too many retries. Quitting.'
            raise e
    
        # See https://dev.twitter.com/docs/error-codes-responses for common codes
    
        if e.e.code == 401:
            print >> sys.stderr, 'Encountered 401 Error (Not Authorized)'
            return None
        elif e.e.code == 404:
            print >> sys.stderr, 'Encountered 404 Error (Not Found)'
            return None
        elif e.e.code == 429: 
            print >> sys.stderr, 'Encountered 429 Error (Rate Limit Exceeded)'
            if sleep_when_rate_limited:
                print >> sys.stderr, "Retrying in 15 minutes...ZzZ..."
                sys.stderr.flush()
                time.sleep(60*15 + 5)
                print >> sys.stderr, '...ZzZ...Awake now and trying again.'
                return 2
            else:
                raise e # Caller must handle the rate limiting issue
        elif e.e.code in (500, 502, 503, 504):
            print >> sys.stderr, 'Encountered %i Error. Retrying in %i seconds' % \
                (e.e.code, wait_period)
            time.sleep(wait_period)
            wait_period *= 1.5
            return wait_period
        else:
            raise e

    # End of nested helper function
    
    wait_period = 2 
    error_count = 0 

    while True:
        try:
            return twitter_api_func(*args, **kw)
        except twitter.api.TwitterHTTPError, e:
            error_count = 0 
            wait_period = handle_twitter_http_error(e, wait_period)
            if wait_period is None:
                return
        except URLError, e:
            error_count += 1
            time.sleep(wait_period)
            wait_period *= 1.5
            print >> sys.stderr, "URLError encountered. Continuing."
            if error_count > max_errors:
                print >> sys.stderr, "Too many consecutive errors...bailing out."
                raise
        except BadStatusLine, e:
            error_count += 1
            time.sleep(wait_period)
            wait_period *= 1.5
            print >> sys.stderr, "BadStatusLine encountered. Continuing."
            if error_count > max_errors:
                print >> sys.stderr, "Too many consecutive errors...bailing out."
                raise

def get_friends_followers_ids(twitter_api, screen_name=None, user_id=None,
                              friends_limit=maxint, followers_limit=maxint):
    
    # Must have either screen_name or user_id (logical xor)
    assert (screen_name != None) != (user_id != None), \
    "Must have screen_name or user_id, but not both"
    
    # See https://dev.twitter.com/docs/api/1.1/get/friends/ids and
    # https://dev.twitter.com/docs/api/1.1/get/followers/ids for details
    # on API parameters
    
    get_friends_ids = partial(make_twitter_request, twitter_api.friends.ids, 
                              count=5000)
    get_followers_ids = partial(make_twitter_request, twitter_api.followers.ids, 
                                count=5000)

    friends_ids, followers_ids = [], []
    
    for twitter_api_func, limit, ids, label in [
                    [get_friends_ids, friends_limit, friends_ids, "friends"], 
                    [get_followers_ids, followers_limit, followers_ids, "followers"]
                ]:
        
        if limit == 0: continue
        
        cursor = -1
        while cursor != 0:
        
            # Use make_twitter_request via the partially bound callable...
            if screen_name: 
                response = twitter_api_func(screen_name=screen_name, cursor=cursor)
            else: # user_id
                response = twitter_api_func(user_id=user_id, cursor=cursor)

            if response is not None:
                ids += response['ids']
                cursor = response['next_cursor']
        
            print >> sys.stderr, 'Fetched {0} total {1} ids for {2}'.format(len(ids), 
                                                    label, (user_id or screen_name))
        
            # XXX: You may want to store data during each iteration to provide an 
            # an additional layer of protection from exceptional circumstances
        
            if len(ids) >= limit or response is None:
                break

    # Do something useful with the IDs, like store them to disk...
    return friends_ids[:friends_limit], followers_ids[:followers_limit]

# Sample usage

twitter_api = oauth_login()

friends_ids, followers_ids = get_friends_followers_ids(twitter_api, 
                                                       screen_name="sb14body", 
                                                       friends_limit=20, 
                                                       followers_limit=20)

Fetched 41 total friends ids for sb14body
Fetched 5000 total followers ids for sb14body


Step 2, get the user profiles by user ID; fetch out the screen name; print out them in table format

In [4]:
def get_user_profile(twitter_api, screen_names=None, user_ids=None):
   
    # Must have either screen_name or user_id (logical xor)
    assert (screen_names != None) != (user_ids != None), \
    "Must have screen_names or user_ids, but not both"
    
    items_to_info = {}

    items = screen_names or user_ids
    
    while len(items) > 0:

        # Process 100 items at a time per the API specifications for /users/lookup.
        # See https://dev.twitter.com/docs/api/1.1/get/users/lookup for details.
        
        items_str = ','.join([str(item) for item in items[:100]])
        items = items[100:]

        if screen_names:
            response = make_twitter_request(twitter_api.users.lookup, 
                                            screen_name=items_str)
        else: # user_ids
            response = make_twitter_request(twitter_api.users.lookup, 
                                            user_id=items_str)
    
        for user_info in response:
            if screen_names:
                items_to_info[user_info['screen_name']] = user_info
            else: # user_ids
                items_to_info[user_info['id']] = user_info

    return items_to_info

twitter_api = oauth_login()
#u = get_user_profile(twitter_api, user_ids=[132373965])
#print u[132373965][u'screen_name']
from prettytable import PrettyTable
y = PrettyTable(["ID", "Screen Name"])
y.padding_width = 1 # One space between column edges and contents (default)
y.align["ID"] = "l"
for ids in followers_ids:
    u = get_user_profile(twitter_api, user_ids=[ids])    
    y.add_row([ids, u[ids][u'screen_name']])
print 'Table of 20 Followers'
print y
print '\n'
x = PrettyTable(["ID", "Screen Name"])
x.padding_width = 1 # One space between column edges and contents (default)
x.align["ID"] = "l"
for ids in friends_ids:
    u = get_user_profile(twitter_api, user_ids=[ids])    
    x.add_row([ids, u[ids][u'screen_name']])
print 'Table of 20 Friends'
print x



Table of 20 Followers
+------------+-----------------+
| ID         |   Screen Name   |
+------------+-----------------+
| 3746448016 |   ericaliu0610  |
| 140997637  |  luans_lovers2  |
| 167634941  | garabatothebest |
| 154571861  |   bexXxdiamond  |
| 3433023903 |   RihanGabriel  |
| 3287722862 | orlandoromaero3 |
| 3641447594 |  bodygoals5ever |
| 2669653387 |    GutiiAlee    |
| 3311797201 |   dudeprashant  |
| 634420959  |   barbarayala1  |
| 3724391368 |  skinnystonerr  |
| 126792537  |    NairDavila   |
| 2783518339 | ashishsharmaba2 |
| 1035982141 |   jmaclauchlan  |
| 941442584  |   MaikonNakata  |
| 477816195  |  robin_herbert  |
| 2727558587 |  adyan_abdullah |
| 3709707017 |    MamaloRan    |
| 3714156934 |   newlanddani   |
| 404685033  |   gauravsinngh  |
+------------+-----------------+


Table of 20 Friends
+------------+-----------------+
| ID         |   Screen Name   |
+------------+-----------------+
| 3236084027 |   apphandouts   |
| 212629710  |  FemaIeTattoos  |

* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [9]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# Sample usage

screen_name = "sb14body"

twitter_api = oauth_login()

friends_ids, followers_ids = get_friends_followers_ids(twitter_api, 
                                                       screen_name=screen_name)
mutual_ids = setwise_friends_followers_analysis(screen_name, friends_ids, followers_ids)

from prettytable import PrettyTable
y = PrettyTable(["ID", "Screen Name"])
y.padding_width = 1 # One space between column edges and contents (default)
y.align["ID"] = "l"
for ids in mutual_ids:
    u = get_user_profile(twitter_api, user_ids = [ids])    
    y.add_row([ids, u[ids][u'screen_name']])
print 'Table of %d Mutual Friends'% len(mutual_ids)
print y


Fetched 41 total friends ids for sb14body
Fetched 5000 total followers ids for sb14body
Fetched 10000 total followers ids for sb14body
Fetched 15000 total followers ids for sb14body
Fetched 20000 total followers ids for sb14body
Fetched 25000 total followers ids for sb14body
Fetched 30000 total followers ids for sb14body
Fetched 35000 total followers ids for sb14body
Fetched 40000 total followers ids for sb14body
Fetched 45000 total followers ids for sb14body
Fetched 48716 total followers ids for sb14body

Table of 12 Mutual Friends
+------------+-----------------+
| ID         |   Screen Name   |
+------------+-----------------+
| 1614029954 |   hahahawhatt   |
| 1369807202 |       TATS      |
| 955425258  |  GoodVibesOnIy  |
| 1478612683 |  FiTGainNTrain  |
| 411553580  |  FascinatingPix |
| 592406989  |    SassyGirI    |
| 212629710  |  FemaIeTattoos  |
| 1572057871 |    LmfaoVine    |
| 730267476  | TrueCookingLove |
| 1342186999 |  illusionposts  |
| 520680283  |      itzTed     |
| 1271233980 |   UberBullshlt  |
+------------+-----------------+


*------------------------

#Problem 4: (optional) Explore the data 

Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API

In [26]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary





*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 10 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

* ** Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through myWPI, in the Assignment "Case Study 1".
        
** Note: Each team just need to submit one submission in myWPI **

# Grading Criteria:

** Totoal Points: 100 **


---------------------------------------------------------------------------
** Notebook:  **
    Points: 60


    -----------------------------------
    Qestion 1:
    Points: 20
    -----------------------------------
    
    (1) Select a topic that you are interested in.
    Points: 6 
    
    (2) Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million. Please check whether the total number of tweets collected is larger than 200?
    Points: 10 
    
    
    (3) Store the tweets you downloaded into a local file (txt file or json file)
    Points: 4 
    
    
    -----------------------------------
    Qestion 2:
    Points: 20
    -----------------------------------
    
    1. Word Count

    (1) Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets.
    Points: 4 

    (2) Plot a table of the top 30 words with their counts 
    Points: 4 
    
    2. Find the most popular tweets in your collection of tweets
    plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.
    Points: 4 
    
    3. Find the most popular Tweet Entities in your collection of tweets

    (1) plot a table of the top 10 hashtags, 
    Points: 4 

    (2) top 10 user mentions that are the most popular in your collection of tweets.
    Points: 4 
    
    
    -----------------------------------
    Qestion 3:
    Points: 20
    -----------------------------------
    
    (1) choose a popular twitter user who has many followers, such as "ladygaga".
    Points: 4 

    (2) Get the list of all friends and all followers of the twitter user.
    Points: 4 

    (3) Plot 20 out of the followers, plot their ID numbers and screen names in a table.
    Points: 4 

    (4) Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.
    Points: 4 
    
    (5) Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table
    Points: 4 

    -----------------------------------
    -----------------------------------
    Problem 4 (Optional): Explore the data
    Bonus points: 10
        Novelty: 5
        Interestingness: 5
    -----------------------------------
    Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API




---------------------------------------------------------------------------
** Report: communicate the results**
    Points: 20

(1) What data you collected?
    Points: 5 

(2) Why this topic is interesting or important to you? (Motivations)
    Points: 5 

(3) How did you analyse the data?
    Points: 5 

(4) What did you find in the data?
(please include figures or tables in the report, but no source code)
    Points: 5 



---------------------------------------------------------------------------
** Slides (for 10 minutes of presentation): Story-telling **
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 

